In [ ]:
# https://www.properati.com.ar/data
! wget -O ar_properties.csv.gz -q https://storage.googleapis.com/properati-data-public/ar_properties.csv.gz
! gzip -d -f ./ar_properties.csv.gz

In [33]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import datetime

# Lectura de datos

In [419]:
df_properties = pd.read_csv('./ar_properties.csv', index_col="id")
df_apred = pd.read_csv('./a_predecir.csv', index_col="id")

In [420]:
df = df_properties.copy()
df_pred = df_apred.copy()

In [31]:
df.head()

,start_date,end_date,created_on,l2,l3,l4,l5,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type,bath_rooms
id,,,,,,,,,,,,,,,,,,,,
LrDZTDP9vyylLQ3SmazY1A==,2020-08-22,2020-09-04,2020-08-22,Capital Federal,Boedo,NaN,NaN,0.0,0.0,0.0,520.0,520.0,70000.0,ARS,Mensual,Depósito con Oficina en Boedo,Depósito con amplias y cómodas oficinas y c...,Oficina,Alquiler,0
taiJHW87e1CmCPpuSgCbBA==,2020-08-22,2020-09-04,2020-08-22,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,NaN,0.0,0.0,2.0,95.0,0.0,35000.0,ARS,Mensual,Oficina - Paseo de la Bahia - Studios I,Excelente oportunidad! Oficina de 95 mts2 con ...,Oficina,Alquiler,1
RqacGyDGe/sKe4TYtuLLtg==,2020-08-22,2020-09-04,2020-08-22,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,NaN,0.0,0.0,2.0,90.0,0.0,27000.0,ARS,Mensual,Oficina - Paseo de la Bahia - Studios I,Excelente oportunidad! Oficina de 90 mts2 con ...,Oficina,Alquiler,1
hbIvfOetTyeiCcFPM4FATA==,2020-08-22,2020-09-04,2020-08-22,Capital Federal,Recoleta,NaN,NaN,0.0,0.0,3.0,265.0,265.0,65000.0,ARS,Mensual,Oficina comercial · 265m² · 2 Cochera,Gran Oportunidad! Unica piso de oficinas de 26...,Oficina,Alquiler,1
srXXAu7Fb8/Kng0/cxUe7g==,2020-08-22,2020-09-04,2020-08-22,Capital Federal,Balvanera,NaN,NaN,0.0,0.0,1.0,80.0,80.0,60000.0,ARS,Mensual,Oficina - Balvanera | Vista al pasaje Discépolo,"Edificio de oficinas desarrollado en Subsuelo,...",Oficina,Alquiler,1


In [565]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1087 entries, 1 to 1087
Data columns (total 76 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   rooms                       1087 non-null   float64
 1   bedrooms                    1087 non-null   float64
 2   bathrooms                   1087 non-null   float64
 3   surface_total               1087 non-null   float64
 4   surface_covered             1087 non-null   float64
 5   price                       1087 non-null   float64
 6   lnrooms                     1087 non-null   float64
 7   lnbedrooms                  1087 non-null   float64
 8   l3_2 Ambientes              1087 non-null   uint8  
 9   l3_Abasto                   1087 non-null   uint8  
 10  l3_Agronomía                1087 non-null   uint8  
 11  l3_Almagro                  1087 non-null   uint8  
 12  l3_Balvanera                1087 non-null   uint8  
 13  l3_Barracas                 1087 non-n

In [564]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102419 entries, U3qdJMKXnOJm0Y1tWpnnfg== to No3is+HhwUGlfgQfzsZbdg==
Data columns (total 76 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   rooms                       102419 non-null  float64
 1   bedrooms                    102419 non-null  float64
 2   bathrooms                   102419 non-null  float64
 3   surface_total               102419 non-null  float64
 4   surface_covered             102419 non-null  float64
 5   price                       102419 non-null  float64
 6   lnrooms                     102419 non-null  float64
 7   lnbedrooms                  102419 non-null  float64
 8   l3_2 Ambientes              102419 non-null  uint8  
 9   l3_Abasto                   102419 non-null  uint8  
 10  l3_Agronomía                102419 non-null  uint8  
 11  l3_Almagro                  102419 non-null  uint8  
 12  l3_Balvanera                102419 n

In [396]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 151918 entries, U3qdJMKXnOJm0Y1tWpnnfg== to RYlWKNYAOPbDbHGoYD/PAA==
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   l3               151342 non-null  object 
 1   rooms            151918 non-null  float64
 2   bedrooms         151918 non-null  float64
 3   bathrooms        151918 non-null  float64
 4   surface_total    151918 non-null  float64
 5   surface_covered  151918 non-null  float64
 6   price            151918 non-null  float64
 7   property_type    151918 non-null  object 
dtypes: float64(6), object(2)
memory usage: 10.4+ MB


# Análisis del conjunto de datos

In [445]:
#analizamos los valores de las distintas variables
df.l3.value_counts()

Palermo                 41131
Belgrano                29278
Recoleta                17910
Caballito               14567
Almagro                 10869
Villa Urquiza           10524
Villa Crespo            10518
Barrio Norte             8843
Nuñez                    7427
Puerto Madero            7050
San Nicolás              6780
Flores                   6694
Balvanera                6538
Colegiales               5018
Retiro                   4958
San Telmo                4401
San Cristobal            4180
Villa Devoto             4055
Monserrat                4012
Saavedra                 3825
Villa del Parque         3394
Floresta                 3064
Boedo                    3048
Barracas                 3021
Mataderos                2811
Liniers                  2615
Congreso                 2249
Parque Chacabuco         2240
Villa Pueyrredón         2110
Villa Luro               2020
Centro / Microcentro     2015
Coghlan                  1972
Chacarita                1882
Parque Pat

In [427]:
df_pred.l4.max()

34.819

In [428]:
df_pred.l4.min()

-38.0152022

# Tratamiento de nulls y outliers

In [430]:
df = df.loc[df["l1"] == "Argentina"]

In [431]:
df = df.loc[df["l2"] == "Capital Federal"]

In [449]:
df_l3 = df[["lon","lat","l3"]]

df_l3.drop(df_l3.loc[df_l3["lon"].isnull()].index,inplace = True)
df_l3.drop(df_l3.loc[df_l3["lat"].isnull()].index,inplace = True)

df_test_l3 = df_l3.loc[df_l3["l3"].isnull()]

df_train_l3 = df_l3.loc[df_l3["l3"].notnull()]

In [508]:
X_l3 = df_train_l3[df_train_l3.columns.drop('l3')]
y_l3 = df_train_l3['l3']

# Partimos en entrenamiento y prueba 

for i in [1,3,5,7,9,11]:
    # Creamos el objeto del modelo
    neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=i)
    
    kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = []
    
    for train, test in kf.split(X_l3):
        
        X_train, X_test, y_train, y_test = X_l3.iloc[train], X_l3.iloc[test], y_l3[train], y_l3[test]
    
        # Entrenamos el modelo
        neigh.fit(X_train, y_train)
    
        # Predecimos
        y_pred_l3 = neigh.predict(X_test)
    
        # Medimos la performance de la predicción
        score = sk.metrics.accuracy_score(y_test, y_pred_l3)
        scores.append(score)
        #print(score)
    print(f"n_{i} {np.mean(scores)}")


n_1 0.8291631320588276
n_3 0.8277355530873358
n_5 0.8295918114585128
n_7 0.8297374037527654
n_9 0.828520119419362
n_11 0.8274888629515317


In [510]:
X_pred_l3 = df_test_l3[df_test_l3.columns.drop('l3')]

# Entrenamiento
neigh = sk.neighbors.KNeighborsClassifier(n_neighbors=7)

# Entrenamos el modelo
neigh.fit(X_train, y_train)

# predicción
df_test_l3["l3"] = neigh.predict(X_pred_l3)

/tmp/ipykernel_17351/4193638472.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_l3["l3"] = neigh.predict(X_pred_l3)


In [511]:
df["l3"].loc[df["l3"].isnull()] = df_test_l3["l3"]

/tmp/ipykernel_17351/553463739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["l3"].loc[df["l3"].isnull()] = df_test_l3["l3"]


In [512]:
df.drop(columns = ["ad_type","l4","l5", "l6", "created_on","start_date", "end_date","l1","l2","price_period","operation_type"], inplace = True )

In [513]:
df.dropna(subset=["price"], inplace = True)

df.dropna(subset=["currency"], inplace = True)

In [514]:
df = df.loc[df["currency"] == "USD"]

df.drop(columns = ["currency"], inplace = True )

/tmp/ipykernel_17351/3524703153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["currency"], inplace = True )


In [515]:
df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)

df = df.loc[df["property_type_check"] == 1]

df.drop(columns = "property_type_check", inplace = True)

/tmp/ipykernel_17351/354315104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["property_type_check"] = df.property_type.apply(lambda x: 1 if x in ("Departamento","PH") else 0)
/tmp/ipykernel_17351/354315104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = "property_type_check", inplace = True)


In [516]:
df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)

/tmp/ipykernel_17351/766927445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.loc[df["rooms"].isnull() & df["bedrooms"].isnull() & df["bathrooms"].isnull() & df["surface_total"].isnull() & df["surface_covered"].isnull()].index, inplace = True)


In [517]:
df_pred["rooms"].fillna(0, inplace = True)
df_pred["bedrooms"].fillna(0, inplace = True)
df_pred["bathrooms"].fillna(0, inplace = True)

In [518]:
df["rooms"].fillna(0, inplace = True)
df["bedrooms"].fillna(0, inplace = True)
df["bathrooms"].fillna(0, inplace = True)

/tmp/ipykernel_17351/764988754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rooms"].fillna(0, inplace = True)
/tmp/ipykernel_17351/764988754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms"].fillna(0, inplace = True)
/tmp/ipykernel_17351/764988754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bathrooms"].fillna(0, inplace = True)


In [519]:
df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]

df["surface_total"].fillna(0, inplace = True)
df["surface_covered"].fillna(0, inplace = True)

/tmp/ipykernel_17351/3354645445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0] = df["surface_covered"].loc[df["surface_total"].isnull() & df["surface_covered"] > 0]
/tmp/ipykernel_17351/3354645445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].fillna(0, inplace = True)
/tmp/ipykernel_17351/3354645445.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [520]:
df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])

/tmp/ipykernel_17351/3611823614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms"].loc[df["bedrooms"] < 0 ] = abs(df["bedrooms"].loc[df["bedrooms"] < 0 ])


In [521]:
df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])

/tmp/ipykernel_17351/3583135774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_total"].loc[df["surface_total"] < 0 ] = abs(df["surface_total"].loc[df["surface_total"] < 0 ])


In [522]:
df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])

/tmp/ipykernel_17351/3604395483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["surface_covered"].loc[df["surface_covered"] < 0 ] = abs(df["surface_covered"].loc[df["surface_covered"] < 0 ])


In [523]:
df.drop(columns = ["title","description"], inplace = True)

/tmp/ipykernel_17351/399705157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["title","description"], inplace = True)


In [566]:
df.drop(columns = ["lat","lon"], inplace = True)

/tmp/ipykernel_17351/1138003077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["lat","lon"], inplace = True)


In [540]:
df.drop(df.loc[df["price"]> 6000000].index, inplace=True)

In [543]:
df.drop(df[df.price < 15000.0].index,inplace = True)

In [544]:
df.drop_duplicates(inplace = True)

In [524]:
df_pred.drop(columns = ["ad_type","l4","l5", "l6", "created_on"], inplace = True )
df_pred.drop(columns = ["l1","l2"], inplace = True )
df_pred.drop(columns = ["currency"], inplace = True )
df_pred.drop(columns = ["price_period"], inplace = True)
df_pred.drop(columns = "operation_type", inplace = True)
df_pred.drop(columns = ["start_date","end_date","services","additionals"], inplace = True)
df_pred.drop(columns = ["title","description","details"], inplace = True)

In [567]:
df_pred.drop(columns = ["lat","lon"], inplace = True)

/tmp/ipykernel_17351/3131608553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred.drop(columns = ["lat","lon"], inplace = True)


# Feature Engineering

In [553]:
df_total = pd.concat([df,df_pred], axis=0)

In [554]:
df_total["lnrooms"] = df_total.rooms.apply(lambda x: x if x == 0 else np.log(x))

In [555]:
df_total["lnbedrooms"] = df_total.bedrooms.apply(lambda x: x if x == 0 else np.log(x))

In [556]:
df_total["lnsurface_covered"] = df_total.surface_covered.apply(lambda x: x if x == 0 else np.log(x))

In [557]:
df_total["lnsurface_total"] = df_total.surface_total.apply(lambda x: x if x == 0 else np.log(x))

In [563]:
df_total["lnsurface_total"].value_counts()

0.000000    20789
3.688879     1672
3.912023     1663
3.806662     1518
3.555348     1478
            ...  
8.858653        1
3.519573        1
8.656259        1
3.832763        1
3.606856        1
Name: lnsurface_total, Length: 1109, dtype: int64

In [558]:
df_total = pd.get_dummies(df_total, columns = ["l3", "property_type"])

KeyError: "None of [Index(['l3', 'property_type'], dtype='object')] are in the [columns]"

In [559]:
df = df_total[df_total.index.isin(df.index)]
df_pred = df_total[df_total.index.isin(df_pred.index)]

# Modelización

In [569]:
X = df[df.columns.drop('price')]
y = df['price']

kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]

In [542]:
X = df[df.columns.drop('price')]
y = df['price']

for n_estimators in [3000, 4000]:
    for max_depth in [10, 12]:     

## Tienen que usar RandomForestRegressor si o si o si. Pueden cambiar los parámetros
#n_estimators = 5000
#max_depth = 5

        reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)

        scores = []
        kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
        for train, test in kf.split(X):
            X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y[train], y[test]
        
            # Entrenamos el modelo
            reg.fit(X_train, y_train)
        
            # Predecimos
            y_pred = reg.predict(X_test)
        
            # Medimos la performance de la predicción
            score = sk.metrics.mean_squared_error(y_test, y_pred)
            scores.append(score)
        print(f"max_depth:{max_depth} - mean: {np.mean(scores)} sd: {np.std(scores)}")
        
    #print(f"n_{n_estimators} - mean: {np.mean(scores)} sd: {np.std(scores)}")

max_depth:10 - mean: 34920465521.80669 sd: 871275656.952177
max_depth:12 - mean: 34149354190.91881 sd: 1143433360.0695686
max_depth:10 - mean: 34916278716.37322 sd: 867722331.4316921


KeyboardInterrupt: 

## Predicción para Kaggle


In [570]:
# quedate con las mismas columnas que ar_properties
X_pred = df_pred[X.columns]

In [571]:
# los mejores hiperparámetros encontrados antes
n_estimators = 4000
max_depth = 10

# Entrenamiento
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1)

# Entrenamos el modelo
reg.fit(X_train, y_train)

# predicción
df_pred["price"] = reg.predict(X_pred)

/tmp/ipykernel_17351/3404264150.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred["price"] = reg.predict(X_pred)


In [572]:
# Grabo la solución
df_pred[["price"]].to_csv("solucion.csv")